<a href="https://colab.research.google.com/github/q1170686276/AndroidReview/blob/master/Image_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using TensorFlow backend

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

sns.set(style='white', context='notebook', palette='deep')

Initial preparation

In [2]:
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

Functions

In [3]:
def get_imlist(path):
  return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]

In [5]:
def convert_to_ela_image(path,quality):
  filename = path
  resaved_filename = filename.split('.')[0] + '.resaved.jpg'
  ELA_filename = filename.split('.')[0] + '.ela.png'

  im = Image.open(filename).convert('RGB')
  im.save(resaved_filename,'JPEG',quality=quality)
  resaved_im = Image.open(resaved_filename)

  ela_im = ImageChops.difference(im, resaved_im)

  extrema = ela_im.getextrema()
  max_diff = max({ex[1] for ex in extrema})
  if max_diff == 0:
    max_diff = 1
  scale = 255.0 / max_diff

  ela_im = ImageEnhance.brightness(ela_im).enhance(scale)
    
  return ela_im


Sample:Real Image

In [6]:
Image.open('')

convert_to_ela_image('',90)

AttributeError: ignored

Sample:Fake image

In [ ]:
Image.open('')

convert_to_ela_image('')


Data preparation

Read dataset and conversion to ELA

In [ ]:
dataset = pd.read_csv('')

In [ ]:
X = []
Y = []

In [ ]:
for index, row in dataset.iterrows():
    X.append(array(convert_to_ela_image(row[0],90),resize((128,128))).flatten() / 255.0)
    Y.appedn(row[1])

Normalization

In [ ]:
X = np.arry(X)
Y = to_categorical(Y,2)

Reshape X

In [ ]:
X = X.reshape(-1,128,128,3)

Train-test split

In [ ]:
X_train,X_val,Y_train,Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32,kernel_size = (5,5),padding = 'valid',
                 activation = 'relu', input_shape = (128,128,3)))
print("Input:",model.input_shape)
print("Output",model.output_shape)

model.add(Conv2D(filters = 32,kernel_size = (5,5),padding = 'valid',
                 activation = 'relu'))
print("Input:",model.input_shape)
print("Output",model.output_shape)

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.25))
print("Input:",model.input_shape)
print("Output",model.output_shape)

model.add(Flatten())
model.add(Dense(256,activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2,activation = "softmax"))

In [ ]:
model.summary()

Add optimizer

In [ ]:
optimizer = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
model.compile(optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

Define early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_acc',
                               min_delta=0,
                               patience=2,
                               verbose=0,mode='auto')

Model training

In [1]:
epochs = 30
batch_size = 100

In [ ]:
history = model.fit(X_train,Y_train,batch_size, epochs =epochs,
                    validation_data = (X_val,Y_val),verbose =2,callbacks=(early_stopping))

Prefprmace measure

Accuracy and loss curves during training-validation

In [ ]:
#Plot the loss. and accuracy curves for training and validation
fig,ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="training loss")
ax[0].plot(history.history['val_loss'],clolr='r', label="validation loss",axes = ax[0])
legend =ax[0].legend(loc='best',shadow=True)

ax[1].plot(history.history['loss'], color='b', label="training loss")
ax[1].plot(history.history['val_loss'],clolr='r', label="validation loss")
legend =ax[1].legend(loc='best',shadow=True)

Confusion matrix

In [ ]:
def plot_confusion_matrix(cm,classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation = 'nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
      cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]

    thresh =cm.max()/1
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
      plt.text(j,i,cm[i,j],
               horizontalaignment="center",
               color="white" if cm[i,j] > thresh else "black")
      
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('preditcted label')
  
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))
